In [1]:
import findspark
findspark.init()

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import to_timestamp, col

In [3]:
spark = SparkSession.builder.appName("projectTest").getOrCreate()

your 131072x1 screen size is bogus. expect trouble
23/09/06 11:33:20 WARN Utils: Your hostname, SUSHAN resolves to a loopback address: 127.0.1.1; using 172.31.68.109 instead (on interface eth0)
23/09/06 11:33:20 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/09/06 11:33:22 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
stationInfo_df = spark.read.csv("data/Fuel_Station_Information.csv", header=True, inferSchema=True)

hourlyPrices_df = spark.read.csv("data/Hourly_Gasoline_Prices.csv", header=True, inferSchema=True)

In [5]:
print(hourlyPrices_df.printSchema(), stationInfo_df.printSchema())

root
 |-- Id: integer (nullable = true)
 |-- isSelf: integer (nullable = true)
 |-- Price: double (nullable = true)
 |-- Date: timestamp (nullable = true)

root
 |-- Id: string (nullable = true)
 |-- Fuel_station_manager: string (nullable = true)
 |-- Petrol_company: string (nullable = true)
 |-- Type: string (nullable = true)
 |-- Station_name: string (nullable = true)
 |-- City: string (nullable = true)
 |-- Latitude: string (nullable = true)
 |-- Longitudine: string (nullable = true)

None None


In [6]:
# Remove Duplicates
hourlyPrices_df = hourlyPrices_df.dropDuplicates()
stationInfo_df = stationInfo_df.dropDuplicates()

In [7]:
# Drop rows with any null values
hourlyPrices_df = hourlyPrices_df.dropna()
stationInfo_df = stationInfo_df.dropna()

In [8]:
stationInfo_df = stationInfo_df.drop("Fuel_station_manager")

In [9]:
hourlyPrices_df = hourlyPrices_df.withColumn("Date", to_timestamp(col("Date"), "yyyy-MM-dd HH:mm:ss"))

In [10]:
hourlyPrices_df.write.parquet("data/cleaned_fuel_prices.parquet")
stationInfo_df.write.parquet("data/cleaned_station_info.parquet")

23/09/06 11:34:16 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers


In [ ]:
# Stop the SparkSession when you're done
spark.stop()